# Chapter 19: Codependence and Regimes (Research Proxy)
AFML focus: dependence structure and regime-conditioned relationships.


In [1]:
import math
import matplotlib.pyplot as plt
import polars as pl
import openquant
import sys
from pathlib import Path
sys.path.insert(0, str(Path('notebooks/python/scripts').resolve()))
from afml_chapter_utils import (
    fetch_panel,
    simple_returns,
    probs_and_sides_from_momentum,
    timestamps_from_dates,
    lag_corr,
    fracdiff_ffd,
)

panel = fetch_panel(window=900)
dates = panel['date'].to_list()
uso = panel['USO'].to_list()
uso_ret = simple_returns(uso)
probs, sides = probs_and_sides_from_momentum(uso)
timestamps = timestamps_from_dates(dates)
asset_names = ['USO', 'BNO', 'XLE', 'GLD', 'UNG']
asset_prices = panel.select(asset_names).rows()
print('rows', panel.height, 'range', dates[0], dates[-1])


rows 900 range 2022-07-08 2026-02-06


In [2]:
bno_ret = simple_returns(panel['BNO'].to_list())
xle_ret = simple_returns(panel['XLE'].to_list())
gld_ret = simple_returns(panel['GLD'].to_list())

corrs = {
    'BNO_l1_to_USO': lag_corr(bno_ret, uso_ret, 1),
    'XLE_l1_to_USO': lag_corr(xle_ret, uso_ret, 1),
    'GLD_l1_to_USO': lag_corr(gld_ret, uso_ret, 1),
}

frontier = openquant.viz.prepare_frontier_payload(
    volatility=[abs(corrs['BNO_l1_to_USO']), abs(corrs['XLE_l1_to_USO']), abs(corrs['GLD_l1_to_USO'])],
    returns=[corrs['BNO_l1_to_USO'], corrs['XLE_l1_to_USO'], corrs['GLD_l1_to_USO']],
    sharpe=[0.0, 0.0, 0.0],
)

plt.figure(figsize=(6,4))
plt.scatter(frontier['x'], frontier['y'])
plt.title('Chapter 19: Dependence Map (Lagged Corr Proxy)')
plt.xlabel('abs corr')
plt.ylabel('corr')
plt.tight_layout()
plt.show()

print(corrs)


{'BNO_l1_to_USO': 0.04334075239184678, 'XLE_l1_to_USO': 0.07701609529111468, 'GLD_l1_to_USO': 0.04347211897072861}


## Interpretation
Dependence structure should be monitored as a dynamic object; regime shifts in cross-asset links can invalidate previously effective features.

